In [1]:
import numpy as np
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import LocallyLinearEmbedding



C:\Users\Huynh\Anaconda3\envs\python3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# read data
X = np.loadtxt('train.dat')
y = np.loadtxt('train.labels')


X_R_test = np.loadtxt('test.dat')


# ilist = []
# for i in range(len(y)):
#     if y[i] == 10 or y[i] == 11: ilist.append(i)
# X = np.delete(X,ilist,0)
# y = np.delete(y,ilist,0)

In [3]:
def make_prediction(model, X_test, y_test):
    y_pred = model.predict(X_test)
    f11 = f1_score(y_test, y_pred, average = 'micro')
    return y_pred, f11

def RFT_classifier(skf, n_estimators=100, max_depth=2, n_components=8, dr='pca'):

    f11_sum = 0
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=0)

    for train_index, test_index in skf.split(X, y):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

#         scale(X_train, copy = False)
#         scale(X_test, copy = False)
#         scaler1 = MinMaxScaler()
#         scaler2 = MinMaxScaler()
#         scaler1.fit_transform(X_train)
#         scaler2.fit_transform(X_test)

        if dr == 'pca':
            pca = PCA(n_components=n_components)
            X_train_reduced = pca.fit_transform(X_train)
            X_test_reduced = pca.fit_transform(X_test)

        elif dr == 'svd':
            svd = TruncatedSVD(n_components=n_components, n_iter=7, random_state=42)
            X_train_reduced = svd.fit_transform(X_train)
            X_test_reduced = svd.fit_transform(X_test)
            
        elif dr == 'lle':
            lle = LocallyLinearEmbedding(n_components=n_components, random_state=42)
            X_train_reduced = lle.fit_transform(X_train)
            X_test_reduced = lle.fit_transform(X_test)
        # model
        model.fit(X_train, y_train)

        # predict
        y_pred, f11 = make_prediction(model, X_test, y_test)
        print('weighted: ', f11)

        f11_sum += f11

    f11 = f11_sum/5
    

    
    return f11, model

In [4]:
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 1)

In [5]:
best_score = 0
best_i_j = 0

n_estimators = [200,400]
max_depth = [300]
n_components = [8]
for i in range(len(n_estimators)):
    for j in range(len(max_depth)):
        for k in range(len(n_components)):
            f11, model = RFT_classifier(skf, n_estimators=n_estimators[i], max_depth=max_depth[j], n_components=n_components[k], dr='svod')
            if f11 >= best_score:
                best_score = f11
                best_i_j_k = [n_estimators[i], max_depth[j], n_components[k]]
            print(best_score, n_estimators[i], max_depth[j], n_components[k])

C:\Users\Huynh\Anaconda3\envs\python3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


weighted:  0.7012497052581939
weighted:  0.6990566037735849
weighted:  0.6948312485248997


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Huynh\Anaconda3\envs\python3\lib\site-packages\IPython\core\interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-b472faa7124e>", line 10, in <module>
    f11, model = RFT_classifier(skf, n_estimators=n_estimators[i], max_depth=max_depth[j], n_components=n_components[k], dr='svod')
  File "<ipython-input-3-0b5fa3718c49>", line 38, in RFT_classifier
    model.fit(X_train, y_train)
  File "C:\Users\Huynh\Anaconda3\envs\python3\lib\site-packages\sklearn\ensemble\forest.py", line 328, in fit
    for i, t in enumerate(trees))
  File "C:\Users\Huynh\Anaconda3\envs\python3\lib\site-packages\sklearn\externals\joblib\parallel.py", line 779, in __call__
    while self.dispatch_one_batch(iterator):
  File "C:\Users\Huynh\Anaconda3\envs\python3\lib\site-packages\sklearn\externals\joblib\parallel.py", line 625, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\Huy

KeyboardInterrupt: 

In [ ]:
# dr = TruncatedSVD(n_components=best_i_j_k[2], n_iter=7, random_state=42)
# # dr = PCA(n_components=best_i_j_k[2])

# X_reduced = dr.fit_transform(X)
# X_R_test_reduced = dr.fit_transform(X_R_test)

In [ ]:
# model = RandomForestClassifier(n_estimators=best_i_j_k[0], max_depth=best_i_j_k[1], random_state=0)
# model.fit(X_reduced, y)

# pred = model.predict(X_R_test_reduced)

# with open("result.dat", "w") as f: 
#     for i in pred:
#         f.write(str(int(i)))
#         f.write('\n')

In [ ]:
# # [220, 30, 8] 666905
# dr = TruncatedSVD(n_components=8, n_iter=7, random_state=42)
# # dr = PCA(n_components=best_i_j_k[2])

# X_reduced = dr.fit_transform(X)
# X_R_test_reduced = dr.fit_transform(X_R_test)

# model = RandomForestClassifier(n_estimators=220, max_depth=30, random_state=0)
# model.fit(X_reduced, y)

# pred = model.predict(X_R_test_reduced)

# with open("result.dat", "w") as f: 
#     for i in pred:
#         f.write(str(int(i)))
#         f.write('\n')

In [ ]:
400 500 704
print(best_i_j_k)


In [ ]:
# [300, 26, 8]
c1=0
c2=0
c3=0
c4=0
c5=0
c6=0
c7=0
c8=0
c9=0
c10=0
c11=0
for i in range(len(y)):
    if y[i] == 1: c1 += 1
    elif y[i] == 2: c2 += 1
    elif y[i] == 3: c3 += 1
    elif y[i] == 4: c4 += 1
    elif y[i] == 5: c5 += 1
    elif y[i] == 6: c6 += 1
    elif y[i] == 7: c7 += 1
    elif y[i] == 8: c8 += 1
    elif y[i] == 9: c9 += 1
    elif y[i] == 10: c10 += 1
    else: c11 += 1
print(c1,c2,c3,c4,c5,c6, c7, c8,c9,c10,c11)

In [ ]:
print (ilist)

In [ ]:
c1=0
c2=0
c3=0
c4=0
c5=0
c6=0
c7=0
c8=0
c9=0
c10=0
c11=0
for i in range(len(y)):
    if y[i] == 1: c1 += 1
    elif y[i] == 2: c2 += 1
    elif y[i] == 3: c3 += 1
    elif y[i] == 4: c4 += 1
    elif y[i] == 5: c5 += 1
    elif y[i] == 6: c6 += 1
    elif y[i] == 7: c7 += 1
    elif y[i] == 8: c8 += 1
    elif y[i] == 9: c9 += 1
    elif y[i] == 10: c10 += 1
    else: c11 += 1
print(c1,c2,c3,c4,c5,c6, c7, c8,c9,c10,c11)